# Coast Corr

### Here you can calculate per-pixel statistics of a imagery catalogue converted to CSV's with environmental monitoring data



In [1]:
from coast_corr import coast_corr, coast_corr_functions
import warnings
#warnings.filterwarnings("ignore")
import ipywidgets
import osmnx as ox
from tkinter import filedialog
from tkinter import *
from ipyleaflet import (
    Map,
    Marker,
    TileLayer, ImageOverlay,
    Polyline, Polygon, Rectangle, Circle, CircleMarker,
    GeoJSON,
    DrawControl, basemaps, basemap_to_tiles
)
import folium
import os
import pandas as pd
import geopandas as gpd
import requests
import numpy as np

## Select folder containing imagery csv files, select satellite type, select environmental parameter:

In [4]:
root = Tk()
root.withdraw()
csv_folder = filedialog.askdirectory()



sat=ipywidgets.Select(
    options=['Sentinel 2', 'Landsat 5', 'Landsat 7', 'Landsat 8','MODIS Aqua','MODIS Terra'],
    value='Sentinel 2',
    # rows=10,
    description='Satellite:',
    disabled=False
)

display(sat)


env_dat=ipywidgets.Select(
    options=['Modeled Wave Energy','River Discharge', 'Ocean Water Level','HF Radar'],
    value='River Discharge',
    description='Environmental Data:',
    disabled=False
)

display(env_dat)


Select(description='Satellite:', options=('Sentinel 2', 'Landsat 5', 'Landsat 7', 'Landsat 8', 'MODIS Aqua', '…

Select(description='Environmental Data:', index=1, options=('Modeled Wave Energy', 'River Discharge', 'Ocean W…

### Here we check to see if you have already combined all of the CSV's of interest to create a pixel value csv. If not, they are combined, if so, they are accessed.

#### In the case that this is your first time combining them, it could take a while...

In [5]:
coast_corr_functions.catalogue_csv(csv_folder)

os.chdir(csv_folder)
band=csv_folder.split("/")[len(csv_folder.split("/"))-1]
path_parent = os.path.dirname(csv_folder)
csv_fold=path_parent+'/coastcorr_csv'
res_out=csv_fold+'/'+band+"_combined_csv.csv"

print("reading combined csv")

sr_df=pd.read_csv(res_out)

Done
reading combined csv


In [ ]:

if env_dat.value == 'River Discharge':
    # from https://stackoverflow.com/questions/38511444/python-download-files-from-google-drive-using-url

    def download_file_from_google_drive(id, destination):
        URL = "https://docs.google.com/uc?export=download"

        session = requests.Session()

        response = session.get(URL, params = { 'id' : id }, stream = True)
        token = get_confirm_token(response)

        if token:
            params = { 'id' : id, 'confirm' : token }
            response = session.get(URL, params = params, stream = True)

        save_response_content(response, destination)    

    def get_confirm_token(response):
        for key, value in response.cookies.items():
            if key.startswith('download_warning'):
                return value

        return None

    def save_response_content(response, destination):
        """
        response = filename for input
        destination = filename for output
        """    
        CHUNK_SIZE = 32768

        with open(destination, "wb") as f:
            for chunk in response.iter_content(CHUNK_SIZE):
                if chunk: # filter out keep-alive new chunks
                    f.write(chunk)



    DATASET_ID = '1Gh9NcqS0g1VI9EPMglbOl3_Cg8MFqPfb'


    destination = '../data.shp'
    download_file_from_google_drive(DATASET_ID, destination)
    env_df= pd.read_csv(destination)
    env_df = gpd.GeoDataFrame(
    env_df, geometry=gpd.points_from_xy(env_df.LNG_GAGE, env_df.LAT_GAGE))
    

     # Make an empty map
    m = folium.Map(location=[35,-124.4], tiles= "Stamen Terrain", zoom_start=4)

    # Show the map
    m

    
    for i in range(0,len(env_df)):
        folium.Marker(
            location=[env_df.iloc[i]['LAT_GAGE'], env_df.iloc[i]['LNG_GAGE']],
            popup=env_df.iloc[i]['STAID'],

           ).add_to(m)
    m

if env_dat.value == 'Modeled Wave Energy':
    # from https://stackoverflow.com/questions/38511444/python-download-files-from-google-drive-using-url

    def download_file_from_google_drive(id, destination):
        URL = "https://docs.google.com/uc?export=download"

        session = requests.Session()

        response = session.get(URL, params = { 'id' : id }, stream = True)
        token = get_confirm_token(response)

        if token:
            params = { 'id' : id, 'confirm' : token }
            response = session.get(URL, params = params, stream = True)

        save_response_content(response, destination)    

    def get_confirm_token(response):
        for key, value in response.cookies.items():
            if key.startswith('download_warning'):
                return value

        return None

    def save_response_content(response, destination):
        """
        response = filename for input
        destination = filename for output
        """    
        CHUNK_SIZE = 32768

        with open(destination, "wb") as f:
            for chunk in response.iter_content(CHUNK_SIZE):
                if chunk: # filter out keep-alive new chunks
                    f.write(chunk)
                    


    DATASET_ID = '11lbhCV-R3dkeioE6RHfiwuLMqhlWNAft'


    destination = '../data.csv'
    download_file_from_google_drive(DATASET_ID, destination)
    env_df= pd.read_csv(destination)
    
    env_df = gpd.GeoDataFrame(
    env_df, geometry=gpd.points_from_xy(env_df.LNG_GAGE, env_df.LAT_GAGE))

     # Make an empty map
    m = folium.Map(location=[35,-124.4], tiles= "Stamen Terrain", zoom_start=4)

    # Show the map
    m
    
    env_df=gpd.clip(env_df, polygon, keep_geom_type=False)


    for i in range(0,len(env_df)):
       folium.Marker(
          location=[env_df.iloc[i]['LAT_GAGE'], env_df.iloc[i]['LNG_GAGE']],
          popup=env_df.iloc[i]['STATID'],

       ).add_to(m)

    
    
    
    
    


station=ipywidgets.Text(
    value='Name of Station',
    placeholder='Type something',
    description='Station:',
    disabled=False
)
display(station)

m

In [ ]:
coast_corr_functions.get_data(env_dat.value,station.value, csv_folder)

if env_dat.value == 'River Discharge':
    collector='USGS'
    site_path=os.path.dirname(os.path.dirname(os.path.dirname(csv_folder)))
    env_fold=site_path+'/environmental_data_csv'
    collec_fold=env_fold+'/'+collector
    file_out= collec_fold+"/"+station.value+'.csv'
    env_df=pd.read_csv(file_out)
    print('downloaded data read')

# Calculate your stats here

In [ ]:
stat_choice=ipywidgets.Select(
    options=['Mean','Median', 'Standard Deviation','Spearman Correlation','Linear Correlation'],
    value='Mean',
    description='Statistic for Composite:',
    disabled=False
)

display(stat_choice)

binning=ipywidgets.Text(
    value='24',
    placeholder='Type something',
    description='Data Bin Hours:',
    disabled=False
)

display(binning)


int0=ipywidgets.Text(
    value='.95',
    placeholder='Type something',
    description='Percentile Upper:',
    disabled=False
)

display(int0)

int1=ipywidgets.Text(
    value='1.0',
    placeholder='Type something',
    description='Percentile Lower:',
    disabled=False
)

display(int1)


## Navigate to source imagery folder

In [ ]:
root = Tk()
root.withdraw()
imgs = filedialog.askdirectory()

In [ ]:
#coast_corr_functions.data_binner(env_df, sr_df, binning.value, station.value, csv_folder, sat.value, env_dat.value)

if env_dat.value=='River Discharge':
        freq=binning.value+'h'
        collector='USGS'
        site_path=os.path.dirname(os.path.dirname(os.path.dirname(csv_folder)))
        env_fold=site_path+'/environmental_data_csv'
        collec_fold=env_fold+'/'+collector
        file_out= collec_fold+"/"+ station.value+'_'+freq+'.csv'
        df_data=pd.read_csv(file_out)
        
        if stat_choice.value== 'Mean':
            points=(df_data.columns[3:len(df_data.columns)-21]).tolist()
            df_fin = pd.DataFrame(columns=['points','stat'])
            i=0
            value=[]
            print('Calculating mean at each position')
            for i in range(0,len(points)):
                try:
                    stat=np.nanmean(df_data.iloc[:,i+3:i+4])
                    value.append(stat)
                except:
                    stat=np.nan
                    value.append(stat)        
                    
        df_fin = pd.DataFrame(columns=['points','stat'])
        df_fin['points']=points
        df_fin['stat']=value
        df_fin['lon']=df_fin.points.str.split('(', expand=True)[1].str.split(' ',expand=True)[0]
        df_fin['lat']=df_fin.points.str.split('(', expand=True)[1].str.split(' ',expand=True)[1].str.split(')', expand=True)[0]
        df_fin['lat']=df_fin['lat'].astype(float)
        df_fin=df_fin[df_fin['lat']>0]

        
        

In [ ]:



raster_creater(df_fin,imgs,csv_folder,env_dat.value,stat_choice.value,int0.value,int1.value)
